In [20]:
# To ensure Chinese characters are displayed correctly
options(encoding = "UTF-8")
Sys.setlocale("LC_CTYPE", "zh_TW.UTF-8")

[1] "zh_TW.UTF-8"

# Date/Time

- [textbook 4.2.3](https://tpemartin.github.io/NTPU-R-for-Data-Science-EN/operations-on-atomic-vectors.html#datetime)

Download flightsData_week8.rds

In [2]:
flightsData <- readRDS("data/flightsData_week8.rds")

In [3]:
dplyr::glimpse(flightsData)

Rows: 4,941
Columns: 24
$ AirlineID          <fct> 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 3U, 5J,~
$ ScheduleStartDate  <chr> "2023-10-13", "2023-10-20", "2023-10-27", "2023-10-~
$ ScheduleEndDate    <chr> "2023-10-15", "2023-10-22", "2023-10-27", "2023-10-~
$ FlightNumber       <chr> "3U3783", "3U3783", "3U3783", "3U3784", "3U3784", "~
$ DepartureAirportID <chr> "CKG", "CKG", "CKG", "TSA", "TSA", "TSA", "TFU", "T~
$ DepartureTime      <chr> "15:00", "15:00", "15:00", "19:00", "19:00", "19:00~
$ CodeShare          <list> [<data.frame[0 x 0]>], [<data.frame[0 x 0]>], [<da~
$ ArrivalAirportID   <chr> "TSA", "TSA", "TSA", "CKG", "CKG", "CKG", "TSA", "T~
$ ArrivalTime        <chr> "18:00", "18:00", "18:00", "22:15", "22:15", "22:15~
$ Monday             <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, TRUE, TRU~
$ Tuesday            <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA~
$ Wednesday          <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, TRUE, TRU~
$ Thursday      

- DepartureTime/ArrivalTime: Some of them has date change "+1" at the end.   
- To parse **Time**, we need date/time/timzone. 
  - date, time: paste date and time together to fit into `lubridate::ymd_hm` format. 
  - Be careful some date need to plus 1 day to fit into the Departure/Arrival Airport's local time expression

Take `DepartureTime` as an example. We need to

1. Find out which `ScheduleStartDate` needs to plus one day, and plus one day on those `ScheduleStartDate`.
2. Paste `ScheduleStartDate` and `DepartureTime` together. 
3. Parse the string into `Data/Time` class with time zone specification

In [4]:
# 1. which ScheduleStartDate need plus 1
## Find out which DepartureTime has +1 at the end
whichNeedPlus1 <- grep(pattern = "+1", flightsData$DepartureTime, fixed = TRUE)

whichNeedPlus1

[1]  475  476  477  478  479  480  492  493  494  516  517  518 3346 3347 3348

- `fixed=TRUE` means the pattern is fixed as the string specified, i.e. "+1" is a fixed pattern no other possible interpretation. This option must always be there until you learn Regular Expression (which will no be taught at this level course).

In [5]:
## Add one day to those ScheduleStartDate

### teach R understand date so it can do plus one day later
flightsData$ScheduleStartDate <- lubridate::ymd(flightsData$ScheduleStartDate)

In [6]:
departureDate <- flightsData$ScheduleStartDate
departureDate[whichNeedPlus1] <- departureDate[whichNeedPlus1] + lubridate::days(1)

In [7]:
# 2. paste date time together
departureTimeString <- flightsData$DepartureTime
# remove "+1" at the end
departureTimeString <- flightsData$DepartureTime
departureTimeString <- sub("+1", "", departureTimeString, fixed = T)

departureTime <- paste(departureDate, departureTimeString)

head(departureTime)


[1] "2023-10-13 15:00" "2023-10-20 15:00" "2023-10-27 15:00" "2023-10-13 19:00"
[5] "2023-10-20 19:00" "2023-10-27 19:00"

In [8]:
flightsData$DepartureTime <- departureTime # replace the old DepartureTime inside the data frame

In [19]:
names(flightsData)


[1] "AirlineID"          "ScheduleStartDate"  "ScheduleEndDate"   
 [4] "FlightNumber"       "DepartureAirportID" "DepartureTime"     
 [7] "CodeShare"          "ArrivalAirportID"   "ArrivalTime"       
[10] "Monday"             "Tuesday"            "Wednesday"         
[13] "Thursday"           "Friday"             "Saturday"          
[16] "Sunday"             "UpdateTime"         "VersionID"         
[19] "Terminal"           "num_codeShare"      "DepartureTimeZone" 
[22] "ArrivalTimeZone"    "DepartTaiwan"       "ArriveTaiwan"

# Data summary

## Departure Date/Time

In [ ]:
flightsData2$ArriveTaiwan

In [38]:
### Range

flightsData2$DepartureTime |> range() |> lubridate::with_tz("Asia/Taipei")

### ScheduleStartDate 
flightsData2 |> split(flightsData2$ArriveTaiwan) -> split_flightsData2

#### Departure from Taiwan
flightsData2 |>
  dplyr::filter(
    DepartTaiwan 
  ) -> DepartFlights

DepartFlights$ScheduleStartDate |> table() |> sort(d=T) |> head()




[1] "2023-10-09 00:05:00 CST" "2024-01-18 14:30:00 CST"


2023-10-23 2023-10-16 2023-10-09 2023-10-10 2023-10-17 2023-10-24 
       539        531        517        118        110        103 

# Exercise

Do the same procedure on `ArrivalTime`.